In [1]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
import math
import calendar
import statsmodels.formula.api as sm
from sklearn.linear_model import LinearRegression
from datetime import datetime
import matplotlib.pyplot as plt
import scipy.stats as stats
import seaborn; seaborn.set()
%matplotlib inline

import sqlite3, csv
from sqlite3 import Error

In [2]:
df1 = pd.read_csv('data/1993_subset.csv')
df2 = pd.read_csv('data/2008_subset.csv')

In [3]:
df1.head()

,SOURCE_REPORTING_UNIT_NAME,FIRE_YEAR,DISCOVERY_DOY,STAT_CAUSE_CODE,STAT_CAUSE_DESCR,FIRE_SIZE,LATITUDE,LONGITUDE,OWNER_DESCR,bio1,...,bio10,bio11,bio12,bio13,bio14,bio15,bio16,bio17,bio18,bio19
0,Humboldt-Toiyabe National Forest,1993,171,1,Lightning,0.2,38.713333,-119.836667,USFS,42,...,125,-27,974,162,15,67,452,61,65,445
1,Humboldt-Toiyabe National Forest,1993,194,9,Miscellaneous,25.0,39.491667,-119.993333,STATE OR PRIVATE,53,...,142,-23,608,98,13,59,266,53,53,266
2,Humboldt-Toiyabe National Forest,1993,195,1,Lightning,0.1,38.721667,-119.925000,USFS,42,...,125,-27,974,162,15,67,452,61,65,445
3,Humboldt-Toiyabe National Forest,1993,198,4,Campfire,0.1,39.600000,-120.131667,USFS,59,...,148,-22,586,97,12,61,261,47,47,261
4,Humboldt-Toiyabe National Forest,1993,212,1,Lightning,0.1,38.556667,-119.815000,USFS,47,...,130,-24,793,132,15,65,364,56,56,361


In [4]:
df2.head()

,SOURCE_REPORTING_UNIT_NAME,FIRE_YEAR,DISCOVERY_DOY,STAT_CAUSE_CODE,FIRE_SIZE,LATITUDE,LONGITUDE,OWNER_DESCR,bio1,bio2,...,bio10,bio11,bio12,bio13,bio14,bio15,bio16,bio17,bio18,bio19
0,Angeles National Forest,2008,15,9,0.1,34.374722,-118.398889,USFS,157,145,...,235,91,445,87,1,88,250,8,16,233
1,Angeles National Forest,2008,21,7,0.1,34.310833,-118.342222,USFS,171,137,...,232,119,416,88,0,93,239,5,13,227
2,Six Rivers National Forest,2008,25,7,0.1,41.235278,-123.657778,USFS,111,150,...,183,46,1352,243,6,77,692,44,56,655
3,Angeles National Forest,2008,38,2,0.1,34.497500,-118.258056,MISSING/NOT SPECIFIED,141,138,...,227,69,434,82,1,86,238,9,18,223
4,Los Padres National Forest,2008,46,2,2.4,36.019444,-121.531944,STATE OR PRIVATE,116,141,...,162,80,691,131,2,88,362,7,14,361


In [5]:
frames = [df1, df2]
result = pd.concat(frames)

In [6]:
result.head()

,DISCOVERY_DOY,FIRE_SIZE,FIRE_YEAR,LATITUDE,LONGITUDE,OWNER_DESCR,SOURCE_REPORTING_UNIT_NAME,STAT_CAUSE_CODE,STAT_CAUSE_DESCR,bio1,...,bio18,bio19,bio2,bio3,bio4,bio5,bio6,bio7,bio8,bio9
0,171,0.2,1993,38.713333,-119.836667,USFS,Humboldt-Toiyabe National Forest,1,Lightning,42,...,65,445,134,43,6155,217,-89,306,-16,124
1,194,25.0,1993,39.491667,-119.993333,STATE OR PRIVATE,Humboldt-Toiyabe National Forest,9,Miscellaneous,53,...,53,266,153,44,6617,252,-94,346,-23,142
2,195,0.1,1993,38.721667,-119.925000,USFS,Humboldt-Toiyabe National Forest,1,Lightning,42,...,65,445,134,43,6155,217,-89,306,-16,124
3,198,0.1,1993,39.600000,-120.131667,USFS,Humboldt-Toiyabe National Forest,4,Campfire,59,...,47,261,164,45,6734,266,-98,364,-22,148
4,212,0.1,1993,38.556667,-119.815000,USFS,Humboldt-Toiyabe National Forest,1,Lightning,47,...,56,361,140,44,6237,227,-89,316,-14,130


In [7]:
#transform DOY and year data into date
result['fire_date']=pd.to_datetime(result['FIRE_YEAR'] * 1000 + result['DISCOVERY_DOY'], format='%Y%j')

In [8]:
#transform data into year and month
result['year'] = pd.DatetimeIndex(result['fire_date']).year
result['month'] = pd.DatetimeIndex(result['fire_date']).month

In [9]:
from sqlalchemy import create_engine
engine = create_engine('sqlite://', echo=False)

In [10]:
result.to_sql('fire', con=engine, if_exists='replace')

In [11]:
pd.read_sql("SELECT * FROM fire limit 10", engine)

,index,DISCOVERY_DOY,FIRE_SIZE,FIRE_YEAR,LATITUDE,LONGITUDE,OWNER_DESCR,SOURCE_REPORTING_UNIT_NAME,STAT_CAUSE_CODE,STAT_CAUSE_DESCR,...,bio3,bio4,bio5,bio6,bio7,bio8,bio9,fire_date,year,month
0,0,171,0.2,1993,38.713333,-119.836667,USFS,Humboldt-Toiyabe National Forest,1,Lightning,...,43,6155,217,-89,306,-16,124,1993-06-20 00:00:00.000000,1993,6
1,1,194,25.0,1993,39.491667,-119.993333,STATE OR PRIVATE,Humboldt-Toiyabe National Forest,9,Miscellaneous,...,44,6617,252,-94,346,-23,142,1993-07-13 00:00:00.000000,1993,7
2,2,195,0.1,1993,38.721667,-119.925000,USFS,Humboldt-Toiyabe National Forest,1,Lightning,...,43,6155,217,-89,306,-16,124,1993-07-14 00:00:00.000000,1993,7
3,3,198,0.1,1993,39.600000,-120.131667,USFS,Humboldt-Toiyabe National Forest,4,Campfire,...,45,6734,266,-98,364,-22,148,1993-07-17 00:00:00.000000,1993,7
4,4,212,0.1,1993,38.556667,-119.815000,USFS,Humboldt-Toiyabe National Forest,1,Lightning,...,44,6237,227,-89,316,-14,130,1993-07-31 00:00:00.000000,1993,7
5,5,214,3.5,1993,39.735000,-120.080000,STATE OR PRIVATE,Humboldt-Toiyabe National Forest,1,Lightning,...,45,6953,290,-87,377,-9,166,1993-08-02 00:00:00.000000,1993,8
6,6,225,0.1,1993,38.708333,-119.800000,STATE OR PRIVATE,Humboldt-Toiyabe National Forest,1,Lightning,...,45,6702,279,-71,350,9,170,1993-08-13 00:00:00.000000,1993,8
7,7,220,0.2,1993,38.795000,-119.791667,STATE OR PRIVATE,Humboldt-Toiyabe National Forest,8,Children,...,45,6702,279,-71,350,9,170,1993-08-08 00:00:00.000000,1993,8
8,8,227,0.3,1993,38.588333,-119.828333,USFS,Humboldt-Toiyabe National Forest,4,Campfire,...,44,6237,227,-89,316,-14,130,1993-08-15 00:00:00.000000,1993,8
9,9,248,0.1,1993,38.723333,-119.658333,USFS,Humboldt-Toiyabe National Forest,1,Lightning,...,44,6758,269,-85,354,-3,160,1993-09-05 00:00:00.000000,1993,9
